## Actividad 1: Implementación de Red Neuronal sin librería:


In [1]:
import random #generar números aleatorios
import numpy as np #para trabajar con arrays y matrices
from keras.datasets import mnist #conjunto de datos de imágenes de dígitos escritos a mano
from keras.utils import to_categorical #
from sklearn.preprocessing import MinMaxScaler #scalar los datos a un rango
from sklearn.model_selection import train_test_split #dividir los datos en conjuntos de entrenamiento y prueba

# Cargar el dataset MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Se crea una lista de tuplas con los elementos de x e y, luego se toma una muestra aleatoria de 2000 tuplas, se descomprime y con zip se vuelven a separar en elementos de x e y 
X, y = zip(*random.sample(list(zip(X_train, y_train)), 2000))

# Convertir en vector
X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')
X = np.reshape(X, (X.shape[0], -1)) #Aplana las imágenes de 28x28 píxeles en un vector de 784 elementos (28*28 = 784)

# Normalizamos Min-Max
X = MinMaxScaler().fit_transform(X)

# Dividimos la muestra en dos, una para entrenar y otra para testing, testeamos con la misma cantidad que entrenamos.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=123)

# Necesitamos para los modelos de clasificación que y_train sea un valor categórico
y_train_value = y_train
y_train = to_categorical(y_train)

In [2]:
import numpy as np

# Clase base para las capas 
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    #espera a que las demas capas las implementen
    def forward_propagation(self, input):
        raise NotImplementedError
    
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError
        
# Clase para capas densas (fully connected)
class FCLayer(Layer):
    def __init__(self, input_size, output_size, lambda_reg=0):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5
        self.lambda_reg = lambda_reg  #Regularización L2

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output
    #Actualiza los pesos usando el gradiente descendente
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)

        # Regularización L2 que castiga error en los pesos
        weights_error += self.lambda_reg * self.weights

        # Actualizar los parámetros
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error
    
# Clase para Capa de Activación. Junto con la capa densa forman perceptrones, activación a la salida de una capa densa
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error
    
# Clase Red, conecta múltiples capas.
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # agrega una capa
    def add(self, layer):
        self.layers.append(layer)

    #define la función de perdida y su derivada
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime
    #predicción en datos de entrada pasando a través de las capas.
    def predict(self, input_data):
        if input_data.ndim == 1: 
            input_data = np.array([[x] for x in input_data])
        samples = len(input_data)
        result = []
        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
        return result
    #entrenamiento de la red
    def fit(self, x_train, y_train, epochs, learning_rate):
        if x_train[0].ndim == 1:
            x_train = np.array([[x] for x in x_train])
        samples = len(x_train)
        for i in range(epochs):
            err = 0
            for j in range(samples):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                err += self.loss(y_train[j], output) 
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
            err /= samples
            
            # calculamos el error promedio entre nodos de salida.
            err = np.mean(err)
            
            # Imprimimos el error promedio de cada época, más que nada
            # para seguimiento del aprendizaje. 
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))
    # Funciones de Activación, y su correspondiente función derivada. 
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    sig = sigmoid(x)  # Calculamos sigmoid(x) para cada elemento del vector
    return sig * (1 - sig)

def relu(x):
    return np.maximum(0, x)

def relu_prime(x):
    # Versión aproximada de ReLu', porque ReLu no es redivable en x=0
    # La derivada de ReLU es 1 si x > 0, y 0 si x <= 0
    return np.where(x > 0, 1, 0)

# función Leaky ReLU
def leaky_relu(x, alpha=0.01):
    # Si x es positivo, devuelve x; si no, devuelve alpha * x
    return np.where(x > 0, x, alpha * x)

# Implementamos la derivada de Leaky ReLU
def leaky_relu_prime(x, alpha=0.01):
    # La derivada es 1 si x > 0, y alpha si x <= 0
    return np.where(x > 0, 1, alpha)


# Funciones de pérdida y su derivada. 
# Error Cuadrático
def mse(y_true, y_hat):
    return (y_true-y_hat)**2

def mse_prime(y_true, y_hat):
    return 2*(y_hat-y_true)

#generador #que crea mini-batches que entrenan la red en pequeños subconjuntos de los datos.

def mini_batch_generator(X, y, batch_size):
    batch_size = 32
    num_samples = X.shape[0]
    indices = np.arange(num_samples)
    np.random.shuffle(indices)

    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)
        batch_indices = indices[start_idx:end_idx]
        yield X[batch_indices], y[batch_indices]
    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)
        batch_indices = indices[start_idx:end_idx]
        yield X[batch_indices], y[batch_indices](y_hat, epsilon, 1 - epsilon)
    
    return -(y_true * np.log(y_hat) + (1 - y_true) * np.log(1 - y_hat))
# Implementamos la función de error BCE
def bce(y_true, y_hat):
    # Evitamos problemas de logaritmo aplicando un pequeño epsilon
    epsilon = 1e-15
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)
    
    return -(y_true * np.log(y_hat) + (1 - y_true) * np.log(1 - y_hat))
# Implementamos la derivada de BCE
def bce_prime(y_true, y_hat):
    # Evitamos problemas de división por 0
    epsilon = 1e-15
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)

    # Derivada de BCE con respecto a la predicción
    return -(y_true / y_hat) + (1 - y_true) / (1 - y_hat)

In [3]:
def fit(self, X, y, batch_size=32):
    n_samples, n_features = X.shape
    self.weights = np.zeros(n_features)
    self.bias = 0

    for _ in range(self.n_iters):
        indices = np.arange(n_samples)
        np.random.shuffle(indices) #baraja los indices de las muestras
        X_shuffled = X[indices]
        y_shuffled = y[indices]

        # Procesamiento del mini-batch
        for i in range(0, n_samples, batch_size):
            X_batch = X_shuffled[i:i+batch_size]
            y_batch = y_shuffled[i:i+batch_size]

            # Calculamos la salida del modelo, se multiplica el mini-batch por los pesos + el sesgo
            linear_output = np.dot(X_batch, self.weights) + self.bias
            y_predicted = self._unit_step_function(linear_output)

            # Calculamos el error
            error = y_batch - y_predicted

            # Calculamos el gradiente promedio del mini-batch
            dw = np.mean(error * X_batch, axis=0)
            db = np.mean(error)

            # Realizamos el ajuste de pesos, con el gradiente calculado y el learning rate
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

        # Ajuste del último mini-batch, para que todas las muestras sean usadas
        if i + batch_size > n_samples:
            X_batch = X_shuffled[i:]
            y_batch = y_shuffled[i:]

            # Calculamos la salida
            linear_output = np.dot(X_batch, self.weights) + self.bias
            y_predicted = self._unit_step_function(linear_output)

            # Calculamos el error
            error = y_batch - y_predicted

            # Calculamos el gradiente del mini-batch
            dw = np.mean(error * X_batch, axis=0)
            db = np.mean(error)

            # Ajuste de pesos
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

In [4]:
#Reduce la dimensionalidad de las entradas tomando el valor máximo definidos en el pool_size.
class MaxPoolingLayer(Layer):
    def __init__(self, pool_size):
        #El tamaño del pool, es decir, la dimensión de la ventana para realizar el max pooling
        self.pool_size = pool_size

    def forward_propagation(self, input_data):
        self.input = input_data
        #Redimensionamos los datos de entrada (reshape) en bloques del tamaño de la ventana
        self.output = np.max(input_data.reshape(-1, self.pool_size, self.pool_size), axis=1)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        error = np.zeros_like(self.input)
        for i in range(self.pool_size):
            for j in range(self.pool_size):
                error[:, i::self.pool_size, j::self.pool_size] = output_error
        return error
#Reduce la dimensionalidad tomando el promedio de los valores de los bloques
class AveragePoolingLayer(Layer):
    def __init__(self, pool_size):
        self.pool_size = pool_size

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.mean(input_data.reshape(-1, self.pool_size, self.pool_size), axis=1)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        error = np.zeros_like(self.input)
        for i in range(self.pool_size):
            for j in range(self.pool_size):
                error[:, i::self.pool_size, j::self.pool_size] = output_error / (self.pool_size ** 2)
        return error

In [5]:
# Clase NoiseLayer que implementa una capa que agrega ruido a los datos de entrada.
# una pequeña dosis de caos que obliga al modelo a aprender de datos ligeramente diferentes cada vez, para evitar el sobreajuste
class NoiseLayer(Layer):
    def __init__(self, noise_std):
        self.noise_std = noise_std #noise_std determina la cantidad de ruido que se agregará a los datos de entrada.

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = input_data + np.random.normal(0, self.noise_std, size=input_data.shape) #crea valores aleatorios alrededor de 0 con una desviación estándar de noise_std
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return output_error

In [6]:
#Reduce sobreajuste al penalizar coeficientes, eliminando características irrelevantes automáticamente.
class L1RegularizationLayer(Layer):
    def __init__(self, lambda_reg):
        self.lambda_reg = lambda_reg

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = input_data
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        error = output_error
        for i in range(self.input.shape[1]):
            #Si el valor de la entrada es positivo, se agrega lambda_reg al error
            if self.input[0, i] > 0:
                error[0, i] += self.lambda_reg
            elif self.input[0, i] < 0:
                error[0, i] -= self.lambda_reg #se resta lambda_reg del error
        return error

## test

In [7]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Necesitamos identificar cuantos nodos tiene nuestra entrada, y eso depende del tamaño de X.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# Agregamos capas al modelo
model.add(FCLayer(entrada_dim, 128))
model.add(ActivationLayer(relu, relu_prime))
model.add(FCLayer(128, 64))
model.add(ActivationLayer(sigmoid, sigmoid_prime))
model.add(FCLayer(64, 10)) 
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Asignamos función de pérdida
model.use(bce, bce_prime)

# Entrenamos el modelo con datos de entrenamiento
model.fit(X_train, y_train, epochs=20, learning_rate=0.01)

# Usamos el modelo para predecir sobre los datos de prueba (validación)
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('MATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

epoch 1/20   error=0.266525
epoch 2/20   error=0.168264
epoch 3/20   error=0.125138
epoch 4/20   error=0.100078
epoch 5/20   error=0.082120
epoch 6/20   error=0.068167
epoch 7/20   error=0.057069
epoch 8/20   error=0.048194
epoch 9/20   error=0.041070
epoch 10/20   error=0.035277
epoch 11/20   error=0.030589
epoch 12/20   error=0.026726
epoch 13/20   error=0.023521
epoch 14/20   error=0.020862
epoch 15/20   error=0.018652
epoch 16/20   error=0.016792
epoch 17/20   error=0.015209
epoch 18/20   error=0.013832
epoch 19/20   error=0.012623
epoch 20/20   error=0.011543
MATRIZ DE CONFUSIÓN para modelo ANN
[[ 88   0   0   1   1   2   2   1   1   0]
 [  1 117   0   1   0   2   1   0   0   0]
 [  0   1  80   3   1   1   4   1   2   1]
 [  0   0   5  94   1   5   0   3   5   2]
 [  1   0   1   0  84   2   1   2   2  10]
 [  1   0   0   0   1  81   0   1   2   0]
 [  2   0   3   0   0   1  90   0   3   0]
 [  2   0   1   0   1   0   0  81   1   0]
 [  1   4   1   6   2   4   0   0  81   2]
 [  1 

test 2

In [12]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Necesitamos identificar cuantos nodos tiene nuestra entrada, y eso depende del tamaño de X.
entrada_dim = len(X_train[0])

# Crear instancia de Network
model = Network()

# Parámetro regularizados
lambda_reg = 0.001

# Agregamos capas al modelo
model.add(FCLayer(entrada_dim, 128))
model.add(L1RegularizationLayer(lambda_reg=0.01))  # Agregar capa L1
model.add(NoiseLayer(noise_std=0.1))  # Agregar capa Noise
model.add(ActivationLayer(relu, relu_prime))
model.add(FCLayer(128, 64))
model.add(L1RegularizationLayer(lambda_reg=0.01))  
model.add(NoiseLayer(noise_std=0.1))  
model.add(ActivationLayer(sigmoid, sigmoid_prime))
model.add(FCLayer(64, 10))
model.add(L1RegularizationLayer(lambda_reg=0.01))  
model.add(ActivationLayer(sigmoid, sigmoid_prime))

# Asignamos función de pérdida
model.use(bce, bce_prime)

# Entrenamos el modelo con datos de entrenamiento
model.fit(X_train, y_train, epochs=20, learning_rate=0.01)

# Usamos el modelo para predecir sobre los datos de prueba (validación)
y_hat = model.predict(X_test)

# Transformamos la salida en un vector one-hot encoded, es decir 0s y un 1. 
for i in range(len(y_hat)):
    y_hat[i] = np.argmax(y_hat[i][0])

# Reportamos los resultados del modelo
matriz_conf = confusion_matrix(y_test, y_hat)

print('MATRIZ DE CONFUSIÓN para modelo ANN')
print(matriz_conf,'\n')
print('La exactitud de testeo del modelo ANN es: {:.3f}'.format(accuracy_score(y_test,y_hat)))

epoch 1/20   error=0.263571
epoch 2/20   error=0.175913
epoch 3/20   error=0.137749
epoch 4/20   error=0.113862
epoch 5/20   error=0.096847
epoch 6/20   error=0.083976
epoch 7/20   error=0.074251
epoch 8/20   error=0.066281
epoch 9/20   error=0.059400
epoch 10/20   error=0.054284
epoch 11/20   error=0.049268
epoch 12/20   error=0.046270
epoch 13/20   error=0.042640
epoch 14/20   error=0.040235
epoch 15/20   error=0.037943
epoch 16/20   error=0.036226
epoch 17/20   error=0.035045
epoch 18/20   error=0.033750
epoch 19/20   error=0.032382
epoch 20/20   error=0.031627
MATRIZ DE CONFUSIÓN para modelo ANN
[[ 70   0   3   3   1   8   6   2   1   2]
 [  1 116   1   1   0   1   1   1   0   0]
 [  0   3  75   3   0   0   2   4   3   4]
 [  2   2   5  76   1  13   0   8   6   2]
 [  1   0   6   1  67   1   5  10   1  11]
 [  0   0   3   5   1  65   3   1   5   3]
 [  0   0   5   0   2   4  82   3   2   1]
 [  0   1   2   0   3   2   0  71   1   6]
 [  1   6   7   5   1  13   3   2  58   5]
 [  1 